<a href="https://colab.research.google.com/github/DingLi23/NLP_smartdispatch_experiment/blob/main/FastTest_Resnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# !nvidia -smi
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

In [ ]:
# !pip uninstall transformers
!pip install transformers==2.2.2

In [ ]:
pip install tensorflow

In [ ]:
pip install fasttext

In [ ]:
pip install ltp

In [ ]:
import csv
import re
from ltp import LTP
import pandas as pd
from gensim.models import Word2Vec, FastText
from gensim.models.word2vec import LineSentence
import multiprocessing

**Fasttext**

First, cutwords by LTP

Then, train fasttext model

In [ ]:
input_val = '/content/drive/My Drive/NLP_gov/history_data/val_22_Mar_2021.csv'
input_test = '/content/drive/My Drive/NLP_gov/history_data/test_22_Mar_2021.csv'
input_train = '/content/drive/My Drive/NLP_gov/history_data/train_22_Mar_2021.csv'
save_model_path = '/content/drive/My Drive/NLP_gov/history_data/fasttext.bin'

In [ ]:


ltp = LTP()

#1.read rawdata request sentences

def LTP_cutdata(inputfile,col):
  rawdata = pd.read_csv(inputfile,usecols = [col])
  rawdata = rawdata.values.tolist()
  # print(rawdata,type(rawdata))
  count = 0
  word_list = []
  words_list = []
  for sentence in rawdata:
    # print('point1 sentence:',sentence)
    count +=1
    if(count == 1):
      continue
    words, _ = ltp.seg(sentence)
    # print('point2 words:',sentence)
    words_list.append(words)
    for word in words:
      word_list.append(word) 
    if (count % 1000 == 0):
      print(count)

  # print('point3:gengerate word_list:',word_list)
  return(word_list,words_list)

 
#2.Train Fasttext model
def train_fasttextmodel(word_list,outputfile):

  model = FastText(word_list, size=100, window=5, min_count =1, workers=multiprocessing.cpu_count())
  model.save(outputfile)
  return(outputfile)

word_list_val, words_list_val = LTP_cutdata(input_val,col=1)
print('finish val')
word_list_test, words_list_test = LTP_cutdata(input_test,col=1)
print('finish test')
word_list_train, words_list_train = LTP_cutdata(input_train,col=1)
print('finish train')

word_list = word_list_val + word_list_test + word_list_train

train_fasttextmodel(word_list,save_model_path)
print('finish all')

Save cutwords to txt

In [ ]:
def writelines_txt(word_list,outputfile):
  write_file = open(outputfile, 'w', encoding='utf-8')
  count = 0
  for word in word_list:
    count +=1
    if(count == 1):
      continue
    word = str(word)
    write_file.write(word)
    write_file.write('\n')
    if (count % 10000 == 0):
      print(count)
  write_file.close()
  print('finish')

words_list_val_file = '/content/drive/My Drive/NLP_gov/history_data/words_list_val_file.txt'
word_list_val_file = '/content/drive/My Drive/NLP_gov/history_data/word_list_val_file.txt'
writelines_txt(words_list_val,words_list_val_file)
writelines_txt(word_list_val,word_list_val_file)

words_list_test_file = '/content/drive/My Drive/NLP_gov/history_data/words_list_test_file.txt'
word_list_test_file = '/content/drive/My Drive/NLP_gov/history_data/word_list_test_file.txt'
writelines_txt(words_list_test,words_list_test_file)
writelines_txt(word_list_test,word_list_test_file)

words_list_train_file = '/content/drive/My Drive/NLP_gov/history_data/words_list_train_file.txt'
word_list_train_file = '/content/drive/My Drive/NLP_gov/history_data/word_list_train_file.txt'
writelines_txt(words_list_train,words_list_train_file)
writelines_txt(word_list_train,word_list_train_file)

test fasttext model

In [ ]:
model = FastText.load('/content/drive/My Drive/NLP_gov/history_data/fasttext.bin')
vocab = model.wv.vocab
# print(vocab)
print(vocab['年检'].index)

Generate csv includes vec_index, class, cutwords

In [ ]:
output_train = '/content/drive/My Drive/NLP_gov/history_data/train_out.csv'
output_test = '/content/drive/My Drive/NLP_gov/history_data/test_out.csv'
output_val = '/content/drive/My Drive/NLP_gov/history_data/val_out.csv'
save_model_path = '/content/drive/My Drive/NLP_gov/history_data/fasttext.bin'
text_file = '/content/drive/My Drive/NLP_gov/history_data/val.csv'
count = 0
ltp = LTP()
#3.save cut words and classes
def get_vocab(Model, path):
  model = Model.load(path)
  vocab = model.wv.vocab
  return vocab

def cut_sentence(sentence):

  word_list = []
  # words_list = []
  # print(sentence)
  global count
  count +=1
  if (count % 1000 == 0):
    print(count)
  sentence = sentence.split(" ")
  # print('list',sentence)
  words, _ = ltp.seg(sentence)
  # words_list.append(words)
  for word in words:
    word_list.append(word) 
  # print('wordslist',words_list)
  # print('wordlist',word_list)
  return word_list


def sentence_to_word_indexes(sentence, vocab):

  indexes = []
  word_list = cut_sentence(sentence)
  # print(word_list)
  for word in word_list:
    # print('word:::',word)
    for char in word:
      try:
        # print(char)
        index = vocab[char].index
      except:
        continue
      indexes.append(index)
    # print(indexes)
  return str(indexes)


def generate_VecIndex(inputfile,savefile): #,cutwords_list):
  data = pd.read_csv(inputfile)
  new_csv = pd.DataFrame(columns=['orginal sentence','ltp_cutwords', 'class','vec_index'])
  new_csv['class'] = data['class']
  new_csv['orginal sentence'] = data['sentence_no_symbol']
  # new_csv['ltp_cutwords'] = pd.DataFrame(cutwords_list)
  print('finish process1')
  vocab = get_vocab(FastText,save_model_path)
  # print(vocab['公积金'].index)
  print('finish vocab')
  new_csv['vec_index'] = data['sentence_no_symbol'].apply(sentence_to_word_indexes, args=(vocab,))
  print('finish index')
  new_csv.to_csv(savefile, index = False)

#word_list_val, words_list_val = LTP_cutdata(text_file,col=1)

generate_VecIndex(input_val,output_val)#,words_list_val)
print('point1')
generate_VecIndex(input_test,output_test)#,word_list_test)
print('point2')
generate_VecIndex(input_train,output_train)#,words_list_train)
print('point3')

**ResNet Train**

In [ ]:
import json
import keras
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from gensim.models import Word2Vec,FastText
from keras import optimizers
from keras.callbacks import EarlyStopping
from keras.layers import BatchNormalization, Conv2D, Activation, Dropout, add, Input, Dense, Flatten
from keras.models import Model
from keras.utils.np_utils import to_categorical
import keras_metrics as km
from sklearn.metrics import recall_score, precision_score


In [ ]:
# class Metrics(keras.callbacks.Callback):
#     def __init__(self, valid_data):
#         super(Metrics, self).__init__()
#         self.validation_data = valid_data
 
#     def on_epoch_end(self, epoch, logs=None):
#         logs = logs or {}
#         val_predict = np.argmax(self.model.predict(self.validation_data[0]), -1)
#         val_targ = self.validation_data[1]
#         if len(val_targ.shape) == 2 and val_targ.shape[1] != 1:
#             val_targ = np.argmax(val_targ, -1)
 
#         _val_f1 = f1_score(val_targ, val_predict, average='macro')
#         _val_recall = recall_score(val_targ, val_predict, average='macro')
#         _val_precision = precision_score(val_targ, val_predict, average='macro')
 
#         logs['val_f1'] = _val_f1
#         logs['val_recall'] = _val_recall
#         logs['val_precision'] = _val_precision
#         print(" — val_f1: %f — val_precision: %f — val_recall: %f" % (_val_f1, _val_precision, _val_recall))


class DataGenerator(keras.utils.Sequence):
    def __init__(self, dataframe, vectors_of_words, batch_size=128, n_classes=10, shuffle=True):
        self.dataframe = dataframe
        self.indexes = np.arange(self.dataframe.__len__())
        self.n_classes = n_classes
        self.labels = to_categorical(self.dataframe['class'], self.n_classes)
        self.vectors_of_words = vectors_of_words
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        """
        Denotes the number of batches per epoch
        """
        return int(np.floor(self.dataframe.__len__() / self.batch_size))

    def __getitem__(self, index):
        """
        Generate one batch of data
        """
        # Generate indices of the batch
        indices = self.indexes[index * self.batch_size:(index + 1) * self.batch_size]

        # Generate data
        X, y = self.__data_generation(indices)

        return X, y

    def on_epoch_end(self):
        """
        Updates indexes after each epoch
        """
        if self.shuffle:
            np.random.shuffle(self.indexes)

    def __data_generation(self, indices):
        """
        Generates data containing batch_size samples
        """
        # Initialization
        X = []
        y = []

        # Generate data
        for i, idx in enumerate(indices):
            input_indices = json.loads(self.dataframe.iloc[idx, 3])
            vec = self.vectors_of_words[input_indices]
            if vec.__len__() < 100:
                num = 100 - vec.__len__()
                vec = np.append(vec, np.zeros((num, 100)), axis=0)
            elif vec.__len__() > 100:
                vec = vec[:100]
            X.append(vec)
            y.append(self.labels[idx])
        X = np.array(X)
        X = np.reshape(X, [X.shape[0], 100, 100, 1])
        y = np.array(y)
        return X, y


def bn_relu(layer, dropout=0, **params):
    layer = BatchNormalization()(layer)
    layer = Activation(params['conv_activation'])(layer)

    if dropout > 0:
        layer = Dropout(dropout)(layer)
    return layer


def res_block(input, filters, stride, dim_up=False):
    if dim_up:
        shortcut = Conv2D(filters=filters,
                          kernel_size=[1, 1],
                          use_bias=False)(input)
    else:
        shortcut = input

    layer = Conv2D(filters=filters,
                   kernel_size=[1, stride],
                   kernel_initializer='random_uniform',
                   padding='same')(input)
    layer = bn_relu(layer, conv_activation='relu')
    layer = Conv2D(filters=filters,
                   kernel_size=[1, stride],
                   kernel_initializer='random_uniform',
                   padding='same')(layer)
    layer = add([layer, shortcut])
    layer = bn_relu(layer, conv_activation='relu')
    return layer


def network(num):
    input = Input(shape=[100, 100, 1])
    layer = Conv2D(filters=64, kernel_size=[1, 100], strides=[1, 100], kernel_initializer='random_uniform')(input)
    layer = bn_relu(layer, conv_activation='relu')
    layer = res_block(layer, 64, stride=2)
    layer = res_block(layer, 128, stride=4, dim_up=True)
    layer = res_block(layer, 256, stride=8, dim_up=True)
    layer = res_block(layer, 512, stride=16, dim_up=True)
    layer = Flatten()(layer)
    output = Dense(num, activation='softmax')(layer)
    model = Model(inputs=[input], outputs=[output])
    model.summary()
    return model


kinds_of_label = 157
model = network(kinds_of_label)

FastText_models = FastText.load('/content/drive/My Drive/NLP_gov/history_data/fasttext.bin')
vectors = FastText_models.wv.vectors


def custom_loss(y_true, y_pred):
    return keras.losses.categorical_crossentropy(y_true, y_pred, label_smoothing=0.1)


adam = optimizers.Adam(lr=0.001)
model.compile(loss=custom_loss,
              optimizer=adam,
              metrics=['accuracy'])


monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=5, verbose=1, mode='auto')

training_batch_size = 64
test_batch_size = 1

train_set = pd.read_csv(output_train)
eval_set = pd.read_csv(output_val)
test_set = pd.read_csv(output_test)

training_generator = DataGenerator(dataframe=train_set, vectors_of_words=vectors,
                                   batch_size=training_batch_size, n_classes=kinds_of_label)
validation_generator = DataGenerator(dataframe=eval_set, vectors_of_words=vectors,
                                     batch_size=training_batch_size, n_classes=kinds_of_label)
test_generator = DataGenerator(dataframe=test_set, vectors_of_words=vectors,
                               batch_size=test_batch_size, n_classes=kinds_of_label, shuffle=False)

model.fit_generator(generator=training_generator,
                    validation_data=validation_generator,
                    epochs=20,
                    callbacks=[monitor])

score = model.evaluate_generator(generator=test_generator)
predictions = model.predict_generator(test_generator)
print('test loss:', score[0], '; test accuracy:', score[1])
model.save('/content/drive/My Drive/NLP_gov/history_data/RCNN_lr001_withLS01.h5')


In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import recall_score, precision_score
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


output_test = '/content/drive/My Drive/NLP_gov/history_data/test_out.csv'
testdata = pd.read_csv(output_test)
labels = np.array(testdata['class'].values)
label_pred = np.argmax(predictions,axis=1)

con_mat = confusion_matrix(labels, label_pred)
# # ValueError: Classification metrics can't handle a mix of multiclass and continuous-multioutput targets
recall = np.diag(con_mat) / np.sum(con_mat, axis = 1)
precision = np.diag(con_mat) / np.sum(con_mat, axis = 0)
avg_recall = np.mean(recall)
avg_precision = np.mean(precision)
print('avg_precision:',avg_precision,'\n','avg_recall:',avg_recall)

con_mat_pd = pd.DataFrame(con_mat)
con_mat_pd.to_csv('/content/drive/My Drive/NLP_gov/history_data/con_mat.csv', index = True)